## Ablation Expriments

We ran various ablative studies in the [SOMA paper](https://download.is.tue.mpg.de/soma/SOMA_ICCV19.pdf)
and here we show how we had the control on training and runtime settings.

Furthermore, we show how to prepare mocap sequences with controllable noise that we use for evaluation purposes.

We assume you have already gone through the  [Running SOMA on SOMA dataset](src/tutorials/runing_soma_on_soma_dataset.ipynb) tutorial and
that you have the required folder structure in place.

## Preparing MoCap Data With Controlled Noise

In the SOMA paper, we use [HDM05](http://resources.mpi-inf.mpg.de/HDM05/) for validation purposes.
This dataset is separating from actual test datasets, i.e. BMLmovi, BMLrub, KIT, and SOMA Dataset,
to avoid probable overfitting to a certain test dataset.
We use choose
[215 sequences across 4 subjects](https://download.is.tue.mpg.de/soma/tutorials/validation_hdm05_sequences.txt).
First download the
[HDM05 folder structure](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=evaluation_mocaps/original/HDM05.tar.bz2)
and uncompress it under
```` support_files/evaluation_mocaps/original ````
as we created the root directories in [Running SOMA on SOMA dataset](src/tutorials/runing_soma_on_soma_dataset.ipynb) tutorial.

Afterward download [these sequences](https://download.is.tue.mpg.de/soma/tutorials/validation_hdm05_sequences.txt) from the 
[HDM05 project website](http://resources.mpi-inf.mpg.de/HDM05/) 
and
place them in their appropriate folders.

Download the
[HDM05 markerlayout](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=smplx/marker_layouts/HDM05.tar.bz2)
and place it under
```` support_files/marker_layouts ````.

We can corrupt a given clean mocap sequence with various noise types including, marker occlusion, ghost points, and broken trajectories.
For this we use the function *def inject_synthetic_noise_into_real_mocap* and below we demonstrate an example application
on HDM05 sequences that we just prepared.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

In [2]:
import os.path as osp
from glob import glob

from soma.data.corrupt_real_mocap_markers import inject_synthetic_noise_into_real_mocap

soma_work_base_dir = '/home/nghorbani/Desktop/SOMA_Test'
support_base_dir = osp.join(soma_work_base_dir, 'support_files')

mocap_dir = osp.join(support_base_dir, 'evaluation_mocaps/original')
mocap_out_base_dir = osp.join(support_base_dir, 'evaluation_mocaps/with_synthetic_noise')

hdm05_marker_layout_fname = osp.join(support_base_dir, 'marker_layouts/HDM05/dg/HDM_dg_01-04_01_120_smplx_finetuned.json')
assert osp.exists(hdm05_marker_layout_fname)

In [3]:
mocap_fnames = glob(osp.join(mocap_dir, 'HDM05', '*/*.c3d'))

args = {
    'marker_layout_fnames': {
        'HDM05': hdm05_marker_layout_fname},
    'dataset_mocap_fnames': {'HDM05': mocap_fnames},
    'oc_g_bt_settings': [(5, 3, 0), (5, 3, 50)], # (# occlusions, # ghost points, # broken trajectories)
    'mocap_unit': 'mm',
    'use_exact_num_oc': False,
    'mocap_rotate': None,
    'fname_filter': ['*bk/*', '*dg/*', '*mm/*', '*tr/*'], #can also be commented out
    'mocap_out_base_dir': mocap_out_base_dir,
}

inject_synthetic_noise_into_real_mocap(**args)

corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:105 -- #215 source mocaps found for HDM05
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:117 -- Starting to process /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/original/HDM05/bk/HDM_bk_01-01_01_120.c3d
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:155 -- Original occ rate 0.00% and with synthetic occlusion 5.01%
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:168 -- Average per-frame ghost rate: 4.91%
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:191 -- Created /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/with_synthetic_noise/HDM05___HDM05___OC_05_G_03_BT_00/bk/HDM_bk_01-01_01_120.pkl, [torch.Size([10283, 43, 3])]


/is/ps3/nghorbani/code-repos/soma_public/src/soma/data/corrupt_real_mocap_markers.py:171: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  label_ids[nan_mask] = nan_class_id


corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:117 -- Starting to process /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/original/HDM05/bk/HDM_bk_01-01_02_120.c3d
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:155 -- Original occ rate 0.00% and with synthetic occlusion 5.01%
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:168 -- Average per-frame ghost rate: 4.89%
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:191 -- Created /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/with_synthetic_noise/HDM05___HDM05___OC_05_G_03_BT_00/bk/HDM_bk_01-01_02_120.pkl, [torch.Size([8747, 43, 3])]
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:117 -- Starting to process /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/original/HDM05/bk/HDM_bk_01-01_03_120.c3d
corrupt_real_mocap_markers:inject_synthetic_noise_into_real_mocap:155 -- Original occ rate 0.00% and

Here we produced two sets of corrupted mocap datasets for validation; i.e. *(5, 3, 0) and (5, 3, 50)*.
The first one does not have broken trajectories, while the latter has up-tp 50 broken point trajectories.
We use the former for per-frame metrics and the latter for tracklet labeling mode.
All ablative experiments in SOMA paper are done in per-frame mode.

# Training SOMA For Ablative Study
Now we get to training a model for our experiment.
Some experiments can share data with the normal HDM05 model because only the model settings are being changed;
i.e. turning off the Transformer model, or using Log-Softmax
instead of the Sinkhorn normalization.
However, some need a new round of data preparation because training data components are being experimented with;
i.e. training stability analysis, no amass marker noise, no CAESAR beta parameters during training.

As mentioned before, due to licensing restrictions we cannot release AMASS marker noise model and CAESAR beta parameters.
However, we can still run the training with these data. Later we will use the pre-trained models.

As an example let's turn off the transformer elements in the model architecture
and replace them with 1D convolutions.

In [4]:
from soma.train.train_soma_multiple import train_multiple_soma

soma_expr_id = 'V48_02_HDM05'

soma_data_settings = [(5, 3, 0.0, 1.0)]

num_gpus = 1
num_cpus = 4

In [5]:
train_multiple_soma(
    soma_data_settings=soma_data_settings,

    soma_train_cfg = {

        'soma.expr_id': f'{soma_expr_id}_TransformerOFF',

        'data_parms.mocap_dataset.marker_layout_fnames': [hdm05_marker_layout_fname],
        'train_parms.batch_size': 256,
        'train_parms.num_workers': num_cpus,

        'model_parms.labeler.enable_transformer': False,
        'dirs.body_dataset_dir': osp.join(support_base_dir, 'smplx/body_dataset'),

        'dirs.support_base_dir': support_base_dir,
        'dirs.work_base_dir': soma_work_base_dir,
        'data_parms.mocap_dataset.amass_marker_noise_model.enable': False, # not released

        'trainer.distributed_backend': 'dp', # in jupyter you cannot use ddp which is the fastest option
        'trainer.num_gpus': num_gpus,
        'trainer.fast_dev_run': True, #comment for full training

        # 'dirs.marker_dataset_dir': '${dirs.work_base_dir}/data/V48_01_HDM05/marker_dataset',
        # 'dirs.amass_marker_noise_dir': '${dirs.work_base_dir}/data/V48_01_HDM05/amass_marker_noise',

    }
)


Global seed set to 100


soma_data_module:__init__:87 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Setting up SOMA data loader
edit_tools:merge_marker_layouts:228 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Merging #1 marker layouts: ['/home/nghorbani/Desktop/SOMA_Test/support_files/marker_layouts/HDM05/dg/HDM_dg_01-04_01_120_smplx_finetuned.json'].
soma_data_module:prepare_training_superset:76 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Created superset at: /home/nghorbani/Desktop/SOMA_Test/data/V48_02_HDM05_TransformerOFF/marker_dataset/superset.json
soma_data_module:__init__:110 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Created /home/nghorbani/Desktop/SOMA_Test/data/V48_02_HDM05_TransformerOFF/marker_dataset/superset.ply
soma_data_module:__init__:126 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- num_points (43) = num_ghost_max (3) * num_body_parts (1) + num_prop_marker_max (0) + num_superset_labels (

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32613/32613 [07:33<00:00, 71.90it/s]


marker_dataset:run:270 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- synthetic data: {'root_orient': (1043609, 1, 3), 'betas': (1043609, 1, 10), 'pose_body': (1043609, 1, 63), 'trans': (1043609, 1, 3), 'markers': (1043609, 1, 120), 'data_is_real': (1043609,)}
marker_dataset:prepare_marker_dataset:358 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- #1043609 synthetic data points created
marker_dataset:prepare_marker_dataset:389 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Not using real data for split train since use_real_data_for (None) does not have this split name
marker_dataset:drop_dict_as_pt:82 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Dumped 1043609 data points as pytorch pt files at: /home/nghorbani/Desktop/SOMA_Test/data/V48_02_HDM05_TransformerOFF/marker_dataset/train
marker_dataset:prepare_marker_dataset:343 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Preparing data files 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4297/4297 [00:59<00:00, 72.57it/s]


marker_dataset:run:270 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- synthetic data: {'root_orient': (137483, 1, 3), 'betas': (137483, 1, 10), 'pose_body': (137483, 1, 63), 'trans': (137483, 1, 3), 'markers': (137483, 1, 120), 'data_is_real': (137483,)}
marker_dataset:prepare_marker_dataset:358 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- #137483 synthetic data points created
marker_dataset:prepare_marker_dataset:389 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Not using real data for split vald since use_real_data_for (None) does not have this split name
marker_dataset:drop_dict_as_pt:82 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Dumped 137483 data points as pytorch pt files at: /home/nghorbani/Desktop/SOMA_Test/data/V48_02_HDM05_TransformerOFF/marker_dataset/vald
marker_dataset:prepare_marker_dataset:426 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- marker_dataset_dir: /home/nghor

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type | Params
------------------------------------
0 | soma_model | SOMA | 2.1 M 
------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.405     Total estimated model params size (MB)


soma_trainer:configure_optimizers:144 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Total trainable gen_params: 2.1014 M.


/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Epoch 0:  50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                             | 1/2 [00:01<00:00,  1.91it/s, loss=472, v_num=, train_loss_step=472.0]
Validating: 0it [00:00, ?it/s]
Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]soma_trainer:validation_epoch_end:253 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- Epoch 0: val_loss:472.74, f1:0.00, acc:0.02
soma_trainer:validation_epoch_end:255 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- lr is ['1.00e-03']
Epoch 0: 100%|█████████████

The above training lasted one epoch and can be turned into a real training  if you comment the line with
```` 'trainer.fast_dev_run': True ````.

Instead, grab, not [grab](https://grab.is.tue.mpg.de/), the
[pre-trained *V48_02_HDM05_TransformerOFF*](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=training_experiments/V48_02_HDM05_TransformerOFF.tar.bz2)
and place it under *training_experiments* replacing the existing *V48_02_HDM05_TransformerOFF* folder.
Also download the [corresponding data](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=smplx/data/V48_01_HDM05.tar.bz2) and place it under
```` data/ ````.


## Running the Evaluation Code
With training out of the way let run the SOMA model on the newly corrupted mocaps.

In [7]:
from soma.tools.run_soma_multiple import run_soma_on_multiple_settings

soma_mocap_target_ds_names = [
    'HDM05___HDM05___OC_05_G_03_BT_00', # without broken trajectories for a per-frame validation
]
soma_data_ids = [
    'OC_05_G_03_real_000_synt_100',
]
soma_expr_ids = [f'{soma_expr_id}_TransformerOFF']

soma_cfg={
    'soma.batch_size': 256,
    'mocap.unit': 'm',
    'save_c3d': True,
    'keep_nan_points': True,  # required for labeling evaluation
    'remove_zero_trajectories': False,  # required for labeling
    'dirs.support_base_dir': support_base_dir,
}

In [11]:
run_soma_on_multiple_settings(soma_expr_ids=soma_expr_ids,
                              soma_mocap_target_ds_names=soma_mocap_target_ds_names,
                              soma_data_ids=soma_data_ids,
                              tracklet_labeling_options=[False],  # always per frame in this experiment
                              soma_cfg=soma_cfg,
                              parallel_cfg={
                                    'max_num_jobs': 10,
                                    'randomly_run_jobs': True,
                                },
                              run_tasks=['soma'],
                              fname_filter=None,
                              # fast_dev_run=True,
                              mocap_ext='.pkl',
                              soma_work_base_dir=soma_work_base_dir,
                              mocap_base_dir=mocap_out_base_dir)


run_soma_multiple:run_soma_on_multiple_settings:245 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_05-03_02_120 -- Submitting SOMA jobs.
parallel_tools:run_parallel_jobs:54 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_05-03_02_120 -- #Job(s) submitted: 215
parallel_tools:run_parallel_jobs:58 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_05-03_02_120 -- max_num_jobs is set to 10. choosing the first #Job(s): 10
parallel_tools:run_parallel_jobs:67 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_05-03_02_120 -- Will run the jobs in random order.
soma_processor:__init__:98 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_01-03_03_120 -- Starting SOMA on /ho

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00,  9.38it/s]

soma_processor:apply_soma:234 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_01-03_03_120 -- SOMA was performing at 3674.0 Hz.


soma_processor:label_perframe:337 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_01-03_03_120 -- 82 labels detected
soma_processor:label_perframe:341 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_01-03_03_120 -- Per-frame labeling yielded 48.88% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_01-03_03_120 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_HDM05_TransformerOFF/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap/HDM05___HDM05___OC_05_G_03_BT_00/mm/HDM_mm_01-03_03_120.pkl


/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/moshpp-3.0-py3.7.egg/moshpp/tools/c3d.py:902: UserWarning: missing parameter ANALOG:LABELS
  warnings.warn('missing parameter {}'.format(name))
/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/moshpp-3.0-py3.7.egg/moshpp/tools/c3d.py:902: UserWarning: missing parameter ANALOG:DESCRIPTIONS
  warnings.warn('missing parameter {}'.format(name))


...

soma_processor:apply_soma:234 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_04-01_02_120 -- SOMA was performing at 7535.0 Hz.


soma_processor:label_perframe:337 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_04-01_02_120 -- 83 labels detected
soma_processor:label_perframe:341 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_04-01_02_120 -- Per-frame labeling yielded 48.35% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_04-01_02_120 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_HDM05_TransformerOFF/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap/HDM05___HDM05___OC_05_G_03_BT_00/mm/HDM_mm_04-01_02_120.pkl
soma_processor:run_soma_once:455 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_04-01_02_120 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00, 10.18it/s]

soma_processor:apply_soma:234 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- SOMA was performing at 7671.0 Hz.


soma_processor:label_perframe:337 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- 83 labels detected
soma_processor:label_perframe:341 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- Per-frame labeling yielded 48.30% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_HDM05_TransformerOFF/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap/HDM05___HDM05___OC_05_G_03_BT_00/mm/HDM_mm_03-02_01_120.pkl
soma_processor:run_soma_once:455 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments

As we have set the _max_num_jobs_ to 10, the SOMA processor labeled only 10 sequences. You can either comment this line to process every mocap or continue for now with only 10.
Let's see how the labeling performance of this model was.

In [12]:
run_soma_on_multiple_settings(soma_expr_ids=soma_expr_ids,
                              soma_mocap_target_ds_names=soma_mocap_target_ds_names,
                              soma_data_ids=soma_data_ids,
                              tracklet_labeling_options=[False],  # always per frame in this experiment
                              soma_cfg=soma_cfg,
                              parallel_cfg={
                                    'max_num_jobs': 10,
                                    'randomly_run_jobs': True,
                                },
                              run_tasks=['eval_label'],
                              fname_filter=None,
                              # fast_dev_run=True,
                              mocap_ext='.pkl',
                              soma_work_base_dir=soma_work_base_dir,
                              mocap_base_dir=mocap_out_base_dir)

run_soma_multiple:run_soma_on_multiple_settings:267 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- Submitting SOMA label evaluations jobs.
parallel_tools:run_parallel_jobs:54 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- #Job(s) submitted: 10
parallel_tools:run_parallel_jobs:58 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- max_num_jobs is set to 10. choosing the first #Job(s): 10
parallel_tools:run_parallel_jobs:67 -- V48_02_HDM05_TransformerOFF -- OC_05_G_03_real_000_synt_100 -- HDM05___HDM05___OC_05_G_03_BT_00 -- mm -- HDM_mm_03-02_01_120 -- Will run the jobs in random order.
eval_labeling:evaluate_labeling_once:81 -- Beginning to evaluate labeling performance.
eval_labeling:evaluate_labeling_once:82 -- mocap_rec: /home/nghorbani/Desk

As expected we observe very bad performance when the Transformer elements are removed from the pipeline. You can run similar experiments on any aspect of the SOMA model, data, and training procedure.